# ✍️ Unified Flow: External Customer Matching

This notebook implements the **external customer matching** logic, consolidating and matching `Sellout Customers` data against both the **C4S Accounts** and **C4S Business Partner Relationship (BPR)** tables.

The primary goal is to **assign a unified ID** (`dim_sellout_cust_id`) to the `sellout_customers_int_matched` table records by sequentially applying a series of matching rules.

---

## 💾 Source Dataframes

The notebook reads data from three source tables:

* **`df_C4S`**: C4S Account data (with Google Place IDs already resolved).
* **`df_selout_cust`**: The internally matched `Sellout Customers` table (`031_sellout_customers_int_matched`). This is the primary **source** for matching.
* **`df_BPR`**: C4S Business Partner Relationship dump.

---

## ⚙️ Matching Logic

The process involves two main matching stages using the `apply_matching` function:

### 1. Matching with C4S Accounts (`Matched_with_c4s`)

This stage attempts to match the `Sellout Customers` against the C4S Accounts using four priority-ordered rule sets:

| Priority | Match Type | Source Column(s) | Target Column(s) | Result Column | Description |
| :---: | :---: | :---: | :---: | :---: | :--- |
| **1** | Exact | `vat_number` | `Vat_Reg_No` | `ext_Tax_Number_4_match` | Exact VAT/Registration Number match (Target: `Vat_Reg_No`). |
| **2** | Exact | `vat_number` | `Tax_Number_4` | `ext_vat_number_match` | Exact VAT/Registration Number match (Target: `Tax_Number_4`). |
| **3** | Exact | `place_id` | `place_id` | `ext_place_id_match` | Exact match on the Google Place ID. |
| **4** | Levenstein (Fuzzy) | `customer_name`, `customer_address`, `customer_postal_code` (Exact) | `Name`, `Street`, `Postal_Code` (Exact) | `ext_levenstein_name_address` | Weighted fuzzy match on name and address fields (min score **0.7**). |

### 2. Matching with C4S BPR (`Matched_with_BPR`)

This stage matches `Sellout Customers` against the **Business Partner Relationship** table using an exact match rule:

* **Rule**: Exact match on a **compound key** of `customer_number` (source) vs. `Number_At_WholeSaler` (target) **AND** `wholesaler_id` (source) vs. `Second_Business_Partner_ID` (target).
* **Result Column**: `ext_BPR_match`.

---

## 🔗 Unification and Final Output

### Data Joining

The results from both matching stages (`Matched_with_c4s` and `Matched_with_BPR`) are joined back to the original `Sellout Customers` records on `unique_id`.

### Assigning Unified ID (`get_fn_flowbase`)

The final step is to call the `assign_unified_id` function, which iteratively chains the matches across the following priority columns to resolve clusters and assign a consistent, unified ID (`dim_sellout_cust_id`) to all connected records:

1.  `ext_Tax_Number_4_match`
2.  `ext_vat_number_match`
3.  `ext_place_id_match`
4.  `ext_levenstein_name_address`
5.  `ext_BPR_match`

The resulting DataFrame (`df_final`) is then persisted to the target Delta table: `d_dnst_020_silver.silver_03.032_sellout_customers_ext_matched`.